# 1. 加载数据

In [1]:
import pickle
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K 
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint

f:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("../data/train.csv")

datafile = open('./data.pkl', 'rb')
# X_test will not be used
X_train = pickle.load(datafile)
X_test = pickle.load(datafile)
word_to_index = pickle.load(datafile)
index_to_word = pickle.load(datafile)
word_to_vec_map = pickle.load(datafile)
datafile.close()

# 2. 定义AUC

In [3]:
# 二元分类的 AUC 的计算方式
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
#-----------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    N = K.sum(1 - y_true)
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N
#-----------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    P = K.sum(y_true)
    TP = K.sum(y_pred * y_true)
    return TP/P

# 3. 创建嵌入层，加载预训练的词向量

In [4]:
# 我们使用的预训练的 word embedding 是 40 万个单词的训练结果，它们的特征维数是 50
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    创建一个 Keras 的 Embedding() 层，并且加载之前已经训练好的 embedding
    """
    
    # 词典中单词的个数+1，+1是 keras 模型的训练要求
    vocab_len = len(word_to_index) + 1
    # 获取单词的特征维数，随便找个单词就行了
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    # 将 embedding 矩阵初始化为全 0 的，大小为 (vocab_len, emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # 将 emb_matrix 的行号当做单词的编号，然后将这个单词的 embedding 放到这一行，这样就把预训练的 embedding 加载进来了
    # 注意，由于单词编号是从 1 开始的，所以行 0 是没有 embedding 的，这就是为什么前面要 +1
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # 创建 Keras 的 Embedding 层
    embedding_layer = Embedding(input_dim=vocab_len, output_dim=emb_dim, trainable=True)

    # build embedding layer，在设置 embedding layer 的权重的时候，这一步是必须的
    embedding_layer.build((None,))
    
    # 将 emb_matrix 设置为 embedding_layer 的权重。
    # 到这里为止我们就创建了一个预训练好的 embedding layer
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [5]:
# 创建 word embedding 层
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

# 4. 创建模型

## 创建共享部分

In [6]:
# 其他所有的分类模型可以基于这个函数进行创建
def shared_model(input_shape, word_to_vec_map, word_to_index, emb_layer):
    """
    返回：一个 Keras 的模型
    
    参数:
    input_shape -- MAX_COMMENT_TEXT_SEQ
    word_to_vec_map
    word_to_index
    
    """
    
    embedding_layer = emb_layer
    # 创建输入层，输入的是句子的单词编号列表
    sentence_indices = Input(shape=input_shape, dtype=np.int32)
    # 句子编号列表进入 embedding_layer 之后会返回对应的 embeddings
    embeddings = embedding_layer(sentence_indices)
    
    dr_r = 0.5
    
    X = BatchNormalization()(embeddings)
    X = LSTM(32, return_sequences=True)(X)
    X = Dropout(dr_r)(X)
    X = BatchNormalization()(X)
    X, _, __ = LSTM(32, return_state = True)(X)
    X = Dropout(dr_r)(X)
    
    X = BatchNormalization()(X)
    X = Dense(8, activation='relu')(X)
    X = Dropout(dr_r)(X)
    
    X = BatchNormalization()(X)
#     X = Dense(1, activation='sigmoid')(X)
    
#     model = Model(inputs=sentence_indices, outputs=X)
    
    
    return sentence_indices, X

In [7]:
MAX_COMMENT_TEXT_SEQ = 200
sentence_indices, shared_parts = shared_model((MAX_COMMENT_TEXT_SEQ, ), word_to_vec_map, word_to_index, embedding_layer)

## 构建多任务模型
参考链接：https://keras.io/getting-started/functional-api-guide/

In [8]:
toxic_dense = Dense(10, activation='relu')(shared_parts)
toxic_output = Dense(1, activation='sigmoid')(toxic_dense)

severe_toxic_dense = Dense(10, activation='relu')(shared_parts)
severe_toxic_output = Dense(1, activation='sigmoid')(severe_toxic_dense)

obscene_dense = Dense(10, activation='relu')(shared_parts)
obscene_output = Dense(1, activation='sigmoid')(obscene_dense)

threat_dense = Dense(10, activation='relu')(shared_parts)
threat_output = Dense(1, activation='sigmoid')(threat_dense)

insult_dense = Dense(10, activation='relu')(shared_parts)
insult_output = Dense(1, activation='sigmoid')(insult_dense)

identity_hate_dense = Dense(10, activation='relu')(shared_parts)
identity_hate_output = Dense(1, activation='sigmoid')(identity_hate_dense)

# 5. 训练模型

In [9]:
model = Model(inputs=sentence_indices, outputs=[toxic_output, 
                                                severe_toxic_output, 
                                                obscene_output, 
                                                threat_output, 
                                                insult_output, 
                                                identity_hate_output])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])

In [ ]:
model_dir = './models'
filepath = model_dir + '/model-{epoch:02d}.h5'
checkpoint = ModelCheckpoint(filepath,monitor='val_loss',save_best_only=True, verbose=1)
callbacks_list = [checkpoint]
train_result = model.fit(X_train['comment_text'], 
            [train[['toxic']], train[['severe_toxic']], train[['obscene']], train[['threat']], train[['insult']], train[['identity_hate']]], 
            epochs=10, 
            batch_size=1024, 
            validation_split=0.07, 
            callbacks = callbacks_list,
            verbose=1)

Train on 148401 samples, validate on 11170 samples
Epoch 1/10
148401/148401 [==============================] - 3217s 22ms/step - loss: 4.1433 - dense_2_loss: 0.6728 - dense_3_loss: 0.7128 - dense_4_loss: 0.6802 - dense_5_loss: 0.7389 - dense_6_loss: 0.6569 - dense_7_loss: 0.6817 - dense_2_auc: 0.6174 - dense_3_auc: 0.6493 - dense_4_auc: 0.5487 - dense_5_auc: nan - dense_6_auc: 0.5719 - dense_7_auc: 0.5936 - val_loss: 2.8589 - val_dense_2_loss: 0.4319 - val_dense_3_loss: 0.5012 - val_dense_4_loss: 0.4905 - val_dense_5_loss: 0.4736 - val_dense_6_loss: 0.4853 - val_dense_7_loss: 0.4763 - val_dense_2_auc: 0.5313 - val_dense_3_auc: 0.8592 - val_dense_4_auc: 0.6491 - val_dense_5_auc: nan - val_dense_6_auc: 0.6281 - val_dense_7_auc: 0.6115

Epoch 00001: val_loss improved from inf to 2.85888, saving model to ./models/model-01.h5
Epoch 2/10
148401/148401 [==============================] - 3801s 26ms/step - loss: 2.4665 - dense_2_loss: 0.4048 - dense_3_loss: 0.4467 - dense_4_loss: 0.3994 - dense

In [ ]:
plt.plot(train_result.history['train_loss'])
plt.plot(train_result.history['validation_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()